# Singleton Networks

In [1]:
import qualreas as qr
import os
import copy

In [2]:
qr_path = os.path.join(os.getenv('PYPROJ'), 'qualreas')
alg_dir = os.path.join(qr_path, "Algebras")

## Make a Test Network

In [3]:
test1_net_dict = {
    'name': 'Network Copy Test #1',
    'algebra': 'Extended_Linear_Interval_Algebra',
    'description': 'Testing/Developing network copy functionality',
    'nodes': [
        ['U', ['ProperInterval', 'Point']],
        ['V', ['ProperInterval', 'Point']],
        ['W', ['ProperInterval']],
        ['X', ['Point']]
    ],
    'edges': [
        ['U', 'V', 'B'],
        ['U', 'W', 'B|M|O'],
        ['W', 'V', 'M|O'],
        ['X', 'W', 'D']
    ]
}

In [4]:
test1_net = qr.Network(algebra_path=alg_dir, network_dict=test1_net_dict)

In [5]:
test1_net.summary()


Network Copy Test #1: 4 nodes, 12 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval', 'Point']
    => U: E|PE
    => V: B
    => W: B|M|O
  V:['ProperInterval', 'Point']
    => V: E|PE
    => U: BI
    => W: MI|OI
  W:['ProperInterval']
    => W: E
    => U: BI|MI|OI
    => V: M|O
    => X: DI
  X:['Point']
    => X: PE
    => W: D


In [6]:
test1_net.propagate()
test1_net.summary()


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval', 'Point']
    => U: E|PE
    => V: B
    => W: B|M|O
    => X: B|DI|PFI
  V:['ProperInterval']
    => V: E
    => U: BI
    => W: MI|OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => U: BI|MI|OI
    => V: M|O
    => X: DI
  X:['Point']
    => X: PE
    => W: D
    => U: BI|D|PF
    => V: B|D|PS


In [8]:
test1_net.get_edge_by_names('U','V')

('U', 'V', 'B')

In [9]:
test1_net.to_dict()

{'name': 'Network Copy Test #1',
 'algebra': 'Extended_Linear_Interval_Algebra',
 'description': 'Testing/Developing network copy functionality',
 'nodes': [['U', ['ProperInterval', 'Point']],
  ['V', ['ProperInterval']],
  ['W', ['ProperInterval']],
  ['X', ['Point']]],
 'edges': [['U', 'V', 'B'],
  ['U', 'W', 'B|M|O'],
  ['U', 'X', 'B|DI|PFI'],
  ['V', 'W', 'MI|OI'],
  ['V', 'X', 'BI|DI|PSI'],
  ['W', 'X', 'DI']]}

### Test Short Version of Network Summary

In [10]:
test1_net.summary(show_all=False)


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval', 'Point']
    => U: E|PE
    => V: B
    => W: B|M|O
    => X: B|DI|PFI
  V:['ProperInterval']
    => V: E
    => W: MI|OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => X: DI
  X:['Point']
    => X: PE


## Test Changing Constraint on an Edge

Look at all the edge contraints

In [11]:
for eg in test1_net.edges:
    print(test1_net.edges[eg[0], eg[1]]['constraint'])

E|PE
B
B|M|O
B|DI|PFI
E
BI
MI|OI
BI|DI|PSI
E
BI|MI|OI
M|O
DI
PE
D
BI|D|PF
B|D|PS


Grab the Head (src) and Tail (tgt) of the 3rd edge, above.

In [12]:
src, tgt = list(test1_net.edges)[2]
test1_net.edges[src,tgt]['constraint']

relset(['B', 'M', 'O'])

Change the constraint and look at the result on the edge & its converse.

In [13]:
test1_net.set_constraint(src, tgt, test1_net.algebra.relset('D|M|FI'))

In [14]:
test1_net.edges[src,tgt]['constraint']

relset(['D', 'FI', 'M'])

In [15]:
test1_net.edges[tgt,src]['constraint']

relset(['DI', 'F', 'MI'])

## Test Copy Network

In [16]:
test1_net_copy = test1_net.copy()
#test1_net_copy = qr.copy(test1_net)

In [17]:
test1_net_copy.summary()


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval', 'Point']
    => U: E|PE
    => V: B
    => W: D|FI|M
    => X: B|DI|PFI
  V:['ProperInterval']
    => V: E
    => U: BI
    => W: MI|OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => U: DI|F|MI
    => V: M|O
    => X: DI
  X:['Point']
    => X: PE
    => U: BI|D|PF
    => V: B|D|PS
    => W: D


In [18]:
unique_edges = []
for src,tgt in test1_net.edges:
    if [tgt,src] not in unique_edges:
        unique_edges.append([src,tgt])

for ue in unique_edges:
    cons = test1_net.edges[ue[0],ue[1]]['constraint']
    if len(cons) > 1:
        print(ue)

[TemporalEntity(['ProperInterval', 'Point'] 'U'), TemporalEntity(['ProperInterval', 'Point'] 'U')]
[TemporalEntity(['ProperInterval', 'Point'] 'U'), TemporalEntity(['ProperInterval'] 'W')]
[TemporalEntity(['ProperInterval', 'Point'] 'U'), TemporalEntity(['Point'] 'X')]
[TemporalEntity(['ProperInterval'] 'V'), TemporalEntity(['ProperInterval'] 'W')]
[TemporalEntity(['ProperInterval'] 'V'), TemporalEntity(['Point'] 'X')]


In [19]:
# Create a list of unique edges with multiple constraints
unique_edges = []
for src,tgt in test1_net.edges:
    if [tgt,src] not in unique_edges:
        if len(test1_net.edges[src,tgt]['constraint']) > 1:
            unique_edges.append([src,tgt])

for src, tgt in unique_edges:
    print(test1_net.edges[src,tgt]['constraint'])

E|PE
D|FI|M
B|DI|PFI
MI|OI
BI|DI|PSI


In [20]:
ue0 = unique_edges[0]
ue0

[TemporalEntity(['ProperInterval', 'Point'] 'U'),
 TemporalEntity(['ProperInterval', 'Point'] 'U')]

In [21]:
ue1 = unique_edges[1]
ue1

[TemporalEntity(['ProperInterval', 'Point'] 'U'),
 TemporalEntity(['ProperInterval'] 'W')]

In [22]:
for rel in test1_net.edges[ue0[0], ue0[1]]['constraint']:
    print(rel)

E
PE


In [23]:
for rel in test1_net.edges[ue1[0], ue1[1]]['constraint']:
    print(rel)

D
FI
M


In [24]:
test1_net.algebra.relset("D")

relset(['D'])

In [25]:
for rel in test1_net.edges[ue1[0], ue1[1]]['constraint']:
    print(rel)

D
FI
M


In [26]:
test1_net.set_constraint(ue1[0], ue1[1], net_copy.algebra.relset("D"))
test1_net.summary()

NameError: name 'net_copy' is not defined

In [27]:
net_copies = []
for rel in test1_net.edges[ue1[0], ue1[1]]['constraint']:
    #print(ue1[0], ue1[1], rel)
    net_copy = test1_net.copy()
    net_copy.set_constraint(ue1[0], ue1[1], net_copy.algebra.relset(rel))
    net_copies.append(net_copy)

KeyError: TemporalEntity(['ProperInterval', 'Point'] 'U')